In [3]:
import cv2
import os
import numpy as np
from pathlib import Path
import json
from tqdm import tqdm

class MultiCameraFrameExtractor:
    def __init__(self, dataset_root, output_root, frame_skip=5):
        """
        Initialize Multi-Camera Frame Extractor
        Args:
            dataset_root: Path to dataset root (multiple_cameras_fall_dataset/)
            output_root: Path to output directory for extracted frames
            frame_skip: Extract every nth frame (default: 5)
        """
        self.dataset_root = Path(dataset_root)
        self.output_root = Path(output_root)
        self.frame_skip = frame_skip
        # Create output directories
        self.create_output_structure()

    def create_output_structure(self):
        """Create organized output directory structure"""
        directories = [
            'extracted_frames/fall',
            'extracted_frames/no_fall', 
            'extracted_frames/by_scenario',
            'extracted_frames/by_camera',
            'metadata'
        ]
        for dir_path in directories:
            (self.output_root / dir_path).mkdir(parents=True, exist_ok=True)
        # Create camera-specific directories
        for cam_id in range(1, 9):
            (self.output_root / f'extracted_frames/by_camera/cam{cam_id}').mkdir(exist_ok=True)
        # Create chute-specific directories  
        for chute_id in range(1, 25):
            (self.output_root / f'extracted_frames/by_scenario/chute_{chute_id:02d}').mkdir(exist_ok=True)

    def extract_video_frames(self, video_path, output_dir, prefix="frame"):
        """Extract frames from a single video file"""
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return []
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = 0
        saved_frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % self.frame_skip == 0:
                frame_filename = f"{prefix}_{frame_count:06d}.jpg"
                frame_path = output_dir / frame_filename
                cv2.imwrite(str(frame_path), frame)
                saved_frames.append({
                    'frame_path': str(frame_path),
                    'frame_number': frame_count,
                    'timestamp': frame_count / fps if fps > 0 else 0
                })
            frame_count += 1
        cap.release()
        return {
            'video_path': str(video_path),
            'total_frames': total_frames,
            'extracted_frames': len(saved_frames),
            'fps': fps,
            'resolution': (width, height),
            'saved_frames': saved_frames
        }

    def process_scenario(self, scenario_num):
        """Process all camera angles for a single chute (renamed from scenario)"""
        chute_dir = self.dataset_root / f"chute{scenario_num:02d}"
        if not chute_dir.exists():
            print(f"Warning: Chute directory {chute_dir} not found")
            return None
        scenario_metadata = {
            'scenario_id': scenario_num,
            'is_fall': scenario_num <= 22,
            'cameras': {}
        }
        for cam_id in range(1, 9):
            video_extensions = ['.avi', '.mp4', '.mov', '.mkv']
            video_path = None
            for ext in video_extensions:
                potential_path = chute_dir / f"cam{cam_id}{ext}"
                if potential_path.exists():
                    video_path = potential_path
                    break
            if video_path is None:
                print(f"Warning: Video for chute {scenario_num:02d}, cam{cam_id} not found")
                continue
            scenario_output = self.output_root / f"extracted_frames/by_scenario/chute_{scenario_num:02d}/cam{cam_id}"
            camera_output = self.output_root / f"extracted_frames/by_camera/cam{cam_id}/chute_{scenario_num:02d}"
            scenario_output.mkdir(parents=True, exist_ok=True)
            camera_output.mkdir(parents=True, exist_ok=True)
            prefix = f"chute{scenario_num:02d}_c{cam_id}"
            frame_metadata = self.extract_video_frames(video_path, scenario_output, prefix)
            for frame_info in frame_metadata['saved_frames']:
                src = Path(frame_info['frame_path'])
                dst = camera_output / src.name
                if not dst.exists():
                    try:
                        os.symlink(src, dst)
                    except OSError:
                        cv2.imwrite(str(dst), cv2.imread(str(src)))
            scenario_metadata['cameras'][f'cam{cam_id}'] = frame_metadata
            category = 'fall' if scenario_num <= 22 else 'no_fall'
            category_output = self.output_root / f"extracted_frames/{category}/chute_{scenario_num:02d}_cam{cam_id}"
            category_output.mkdir(parents=True, exist_ok=True)
            for frame_info in frame_metadata['saved_frames']:
                src = Path(frame_info['frame_path'])
                dst = category_output / src.name
                if not dst.exists():
                    try:
                        os.symlink(src, dst)
                    except OSError:
                        cv2.imwrite(str(dst), cv2.imread(str(src)))
        return scenario_metadata

    def process_all_scenarios(self):
        all_metadata = {
            'dataset_info': {
                'total_scenarios': 24,
                'fall_scenarios': 22,
                'no_fall_scenarios': 2,
                'cameras_per_scenario': 8,
                'frame_skip': self.frame_skip
            },
            'scenarios': {}
        }
        print(f"Processing Multiple Cameras Fall Dataset...")
        print(f"Dataset root: {self.dataset_root}")
        print(f"Output root: {self.output_root}")
        print(f"Frame skip: {self.frame_skip}")
        print("="*60)
        for scenario_num in tqdm(range(1, 25), desc="Processing scenarios"):
            scenario_metadata = self.process_scenario(scenario_num)
            if scenario_metadata:
                all_metadata['scenarios'][f'chute_{scenario_num:02d}'] = scenario_metadata
        metadata_file = self.output_root / 'metadata/extraction_metadata.json'
        with open(metadata_file, 'w') as f:
            json.dump(all_metadata, f, indent=2)
        print(f"\nFrame extraction completed!")
        print(f"Metadata saved to: {metadata_file}")
        return all_metadata

    def generate_summary_report(self, metadata):
        total_frames = 0
        total_videos = 0
        fall_frames = 0
        no_fall_frames = 0
        for scenario_id, scenario_data in metadata['scenarios'].items():
            scenario_num = int(scenario_id.split('_')[1])
            is_fall = scenario_num <= 22
            for cam_id, cam_data in scenario_data['cameras'].items():
                total_videos += 1
                extracted_frames = cam_data['extracted_frames']
                total_frames += extracted_frames
                if is_fall:
                    fall_frames += extracted_frames
                else:
                    no_fall_frames += extracted_frames
        report = f"""
MULTI-CAMERA FALL DATASET EXTRACTION REPORT
{'='*60}
Total Scenarios Processed: {len(metadata['scenarios'])}
Total Videos Processed: {total_videos}
Total Frames Extracted: {total_frames:,}
Fall Scenarios: {len([s for s in metadata['scenarios'].keys() if int(s.split('_')[1]) <= 22])}
No-Fall Scenarios: {len([s for s in metadata['scenarios'].keys() if int(s.split('_')[1]) > 22])}
Fall Frames: {fall_frames:,}
No-Fall Frames: {no_fall_frames:,}
Frame Skip Used: {metadata['dataset_info']['frame_skip']}
Average Frames per Video: {total_frames/total_videos:.1f}
Output Structure:
- extracted_frames/fall/          # Fall frames organized by chute+camera
- extracted_frames/no_fall/       # No-fall frames organized by chute+camera  
- extracted_frames/by_scenario/   # Frames organized by chute
- extracted_frames/by_camera/     # Frames organized by camera angle
- metadata/                       # Extraction metadata and logs
        """
        report_file = self.output_root / 'metadata/extraction_report.txt'
        with open(report_file, 'w') as f:
            f.write(report)
        print(report)
        return report

# Example usage
if __name__ == "__main__":
    extractor = MultiCameraFrameExtractor(
        dataset_root="/kaggle/input/multiple-cameras-fall-dataset/dataset/dataset",  # Your provided root path
        output_root="processed_output/",
        frame_skip=5
    )
    metadata = extractor.process_all_scenarios()
    extractor.generate_summary_report(metadata)

Processing Multiple Cameras Fall Dataset...
Dataset root: /kaggle/input/multiple-cameras-fall-dataset/dataset/dataset
Output root: processed_output
Frame skip: 5


Processing scenarios:   0%|          | 0/24 [00:00<?, ?it/s][mpeg4 @ 0x13c9e7c0] ac-tex damaged at 3 19
[mpeg4 @ 0x13c9e7c0] Error at MB: 877
[mpeg4 @ 0x13c86480] ac-tex damaged at 42 21
[mpeg4 @ 0x13c86480] Error at MB: 1008
[mpeg4 @ 0x13c9e7c0] ac-tex damaged at 24 17
[mpeg4 @ 0x13c9e7c0] Error at MB: 806
[mpeg4 @ 0x13c99880] ac-tex damaged at 13 23
[mpeg4 @ 0x13c99880] Error at MB: 1071
[mpeg4 @ 0x13c86500] ac-tex damaged at 8 12
[mpeg4 @ 0x13c86500] Error at MB: 560
[mpeg4 @ 0x141b90c0] ac-tex damaged at 12 17
[mpeg4 @ 0x141b90c0] Error at MB: 794
[mpeg4 @ 0x13cc3400] mcbpc damaged at 37 18
[mpeg4 @ 0x13cc3400] Error at MB: 865
[mpeg4 @ 0x13cc3300] Error at MB: 1285
Processing scenarios:   4%|▍         | 1/24 [00:11<04:25, 11.56s/it][mpeg4 @ 0x141d6f80] Error at MB: 898
[mpeg4 @ 0x13c99a80] ac-tex damaged at 38 16
[mpeg4 @ 0x13c99a80] Error at MB: 774
[mpeg4 @ 0x13cc3100] mcbpc damaged at 30 8
[mpeg4 @ 0x13cc3100] Error at MB: 398
[mpeg4 @ 0x14046a40] ac-tex damaged at 43 21
[mpeg4

KeyboardInterrupt: 

In [4]:
import cv2
import os
import numpy as np
from pathlib import Path
import json
from tqdm import tqdm

class MultiCameraFrameExtractor:
    def __init__(self, dataset_root, output_root, frame_skip=5):
        self.dataset_root = Path(dataset_root)
        self.output_root = Path(output_root)
        self.frame_skip = frame_skip
        self.create_output_structure()

    def create_output_structure(self):
        directories = [
            'extracted_frames/fall',
            'extracted_frames/no_fall',
            'extracted_frames/by_scenario',
            'extracted_frames/by_camera',
            'metadata'
        ]
        for dir_path in directories:
            (self.output_root / dir_path).mkdir(parents=True, exist_ok=True)
        for cam_id in range(1, 9):
            (self.output_root / f'extracted_frames/by_camera/cam{cam_id}').mkdir(exist_ok=True)
        for chute_id in range(1, 25):
            (self.output_root / f'extracted_frames/by_scenario/chute_{chute_id:02d}').mkdir(exist_ok=True)

    def extract_video_frames(self, video_path, output_dir, prefix="frame"):
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return []
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = 0
        saved_frames = []
        consecutive_failures = 0  # To break if too many bad frames in a row
        MAX_FAILURES = 10

        while True:
            ret, frame = cap.read()
            if not ret:
                consecutive_failures += 1
                if consecutive_failures >= MAX_FAILURES:
                    # Too many consecutive failures, break loop
                    break
                else:
                    frame_count += 1
                continue
            else:
                consecutive_failures = 0

            if frame_count % self.frame_skip == 0:
                frame_filename = f"{prefix}_{frame_count:06d}.jpg"
                frame_path = output_dir / frame_filename
                try:
                    cv2.imwrite(str(frame_path), frame)
                    saved_frames.append({
                        'frame_path': str(frame_path),
                        'frame_number': frame_count,
                        'timestamp': frame_count / fps if fps > 0 else 0
                    })
                except Exception as e:
                    print(f"Warning: Could not save frame {frame_count} of video {video_path}: {e}")
            frame_count += 1

        cap.release()
        return {
            'video_path': str(video_path),
            'total_frames': total_frames,
            'extracted_frames': len(saved_frames),
            'fps': fps,
            'resolution': (width, height),
            'saved_frames': saved_frames
        }

    def process_scenario(self, scenario_num):
        chute_dir = self.dataset_root / f"chute{scenario_num:02d}"
        if not chute_dir.exists():
            print(f"Warning: Chute directory {chute_dir} not found")
            return None
        scenario_metadata = {
            'scenario_id': scenario_num,
            'is_fall': scenario_num <= 22,
            'cameras': {}
        }
        for cam_id in range(1, 9):
            video_extensions = ['.avi', '.mp4', '.mov', '.mkv']
            video_path = None
            for ext in video_extensions:
                potential_path = chute_dir / f"cam{cam_id}{ext}"
                if potential_path.exists():
                    video_path = potential_path
                    break
            if video_path is None:
                print(f"Warning: Video for chute {scenario_num:02d}, cam{cam_id} not found")
                continue
            scenario_output = self.output_root / f"extracted_frames/by_scenario/chute_{scenario_num:02d}/cam{cam_id}"
            camera_output = self.output_root / f"extracted_frames/by_camera/cam{cam_id}/chute_{scenario_num:02d}"
            scenario_output.mkdir(parents=True, exist_ok=True)
            camera_output.mkdir(parents=True, exist_ok=True)
            prefix = f"chute{scenario_num:02d}_c{cam_id}"
            frame_metadata = self.extract_video_frames(video_path, scenario_output, prefix)
            for frame_info in frame_metadata['saved_frames']:
                src = Path(frame_info['frame_path'])
                dst = camera_output / src.name
                if not dst.exists():
                    try:
                        os.symlink(src, dst)
                    except OSError:
                        cv2.imwrite(str(dst), cv2.imread(str(src)))
            scenario_metadata['cameras'][f'cam{cam_id}'] = frame_metadata
            category = 'fall' if scenario_num <= 22 else 'no_fall'
            category_output = self.output_root / f"extracted_frames/{category}/chute_{scenario_num:02d}_cam{cam_id}"
            category_output.mkdir(parents=True, exist_ok=True)
            for frame_info in frame_metadata['saved_frames']:
                src = Path(frame_info['frame_path'])
                dst = category_output / src.name
                if not dst.exists():
                    try:
                        os.symlink(src, dst)
                    except OSError:
                        cv2.imwrite(str(dst), cv2.imread(str(src)))
        return scenario_metadata

    def process_all_scenarios(self):
        all_metadata = {
            'dataset_info': {
                'total_scenarios': 24,
                'fall_scenarios': 22,
                'no_fall_scenarios': 2,
                'cameras_per_scenario': 8,
                'frame_skip': self.frame_skip
            },
            'scenarios': {}
        }
        print(f"Processing Multiple Cameras Fall Dataset...")
        print(f"Dataset root: {self.dataset_root}")
        print(f"Output root: {self.output_root}")
        print(f"Frame skip: {self.frame_skip}")
        print("="*60)
        for scenario_num in tqdm(range(1, 25), desc="Processing scenarios"):
            scenario_metadata = self.process_scenario(scenario_num)
            if scenario_metadata:
                all_metadata['scenarios'][f'chute_{scenario_num:02d}'] = scenario_metadata
        metadata_file = self.output_root / 'metadata/extraction_metadata.json'
        with open(metadata_file, 'w') as f:
            json.dump(all_metadata, f, indent=2)
        print(f"\nFrame extraction completed!")
        print(f"Metadata saved to: {metadata_file}")
        return all_metadata

    def generate_summary_report(self, metadata):
        total_frames = 0
        total_videos = 0
        fall_frames = 0
        no_fall_frames = 0
        for scenario_id, scenario_data in metadata['scenarios'].items():
            scenario_num = int(scenario_id.split('_')[1])
            is_fall = scenario_num <= 22
            for cam_id, cam_data in scenario_data['cameras'].items():
                total_videos += 1
                extracted_frames = cam_data['extracted_frames']
                total_frames += extracted_frames
                if is_fall:
                    fall_frames += extracted_frames
                else:
                    no_fall_frames += extracted_frames
        report = f"""
MULTI-CAMERA FALL DATASET EXTRACTION REPORT
{'='*60}
Total Scenarios Processed: {len(metadata['scenarios'])}
Total Videos Processed: {total_videos}
Total Frames Extracted: {total_frames:,}
Fall Scenarios: {len([s for s in metadata['scenarios'].keys() if int(s.split('_')[1]) <= 22])}
No-Fall Scenarios: {len([s for s in metadata['scenarios'].keys() if int(s.split('_')[1]) > 22])}
Fall Frames: {fall_frames:,}
No-Fall Frames: {no_fall_frames:,}
Frame Skip Used: {metadata['dataset_info']['frame_skip']}
Average Frames per Video: {total_frames / total_videos:.1f}
Output Structure:
- extracted_frames/fall/          # Fall frames organized by chute+camera
- extracted_frames/no_fall/       # No-fall frames organized by chute+camera  
- extracted_frames/by_scenario/   # Frames organized by chute
- extracted_frames/by_camera/     # Frames organized by camera angle
- metadata/                       # Extraction metadata and logs
        """
        report_file = self.output_root / 'metadata/extraction_report.txt'
        with open(report_file, 'w') as f:
            f.write(report)
        print(report)
        return report

if __name__ == "__main__":
    extractor = MultiCameraFrameExtractor(
        dataset_root="/kaggle/input/multiple-cameras-fall-dataset/dataset/dataset",
        output_root="processed_output/",
        frame_skip=5
    )
    metadata = extractor.process_all_scenarios()
    extractor.generate_summary_report(metadata)


Processing Multiple Cameras Fall Dataset...
Dataset root: /kaggle/input/multiple-cameras-fall-dataset/dataset/dataset
Output root: processed_output
Frame skip: 5


Processing scenarios:   0%|          | 0/24 [00:00<?, ?it/s][mpeg4 @ 0x1425c9c0] ac-tex damaged at 3 19
[mpeg4 @ 0x1425c9c0] Error at MB: 877
[mpeg4 @ 0x14046d40] ac-tex damaged at 42 21
[mpeg4 @ 0x14046d40] Error at MB: 1008
[mpeg4 @ 0x13d4b3c0] ac-tex damaged at 24 17
[mpeg4 @ 0x13d4b3c0] Error at MB: 806
[mpeg4 @ 0x14046d40] ac-tex damaged at 13 23
[mpeg4 @ 0x14046d40] Error at MB: 1071
[mpeg4 @ 0x14046d40] ac-tex damaged at 8 12
[mpeg4 @ 0x14046d40] Error at MB: 560
[mpeg4 @ 0x1425c9c0] ac-tex damaged at 12 17
[mpeg4 @ 0x1425c9c0] Error at MB: 794
[mpeg4 @ 0x13fee4c0] mcbpc damaged at 37 18
[mpeg4 @ 0x13fee4c0] Error at MB: 865
[mpeg4 @ 0x13d4b3c0] Error at MB: 1285
Processing scenarios:   4%|▍         | 1/24 [00:30<11:33, 30.16s/it][mpeg4 @ 0x13ecc8c0] Error at MB: 898
[mpeg4 @ 0x14046d40] ac-tex damaged at 38 16
[mpeg4 @ 0x14046d40] Error at MB: 774
[mpeg4 @ 0x13c84800] mcbpc damaged at 30 8
[mpeg4 @ 0x13c84800] Error at MB: 398
[mpeg4 @ 0x13c7e8c0] ac-tex damaged at 43 21
[mpeg4


Frame extraction completed!
Metadata saved to: processed_output/metadata/extraction_metadata.json

MULTI-CAMERA FALL DATASET EXTRACTION REPORT
Total Scenarios Processed: 24
Total Videos Processed: 192
Total Frames Extracted: 52,306
Fall Scenarios: 22
No-Fall Scenarios: 2
Fall Frames: 37,776
No-Fall Frames: 14,530
Frame Skip Used: 5
Average Frames per Video: 272.4
Output Structure:
- extracted_frames/fall/          # Fall frames organized by chute+camera
- extracted_frames/no_fall/       # No-fall frames organized by chute+camera  
- extracted_frames/by_scenario/   # Frames organized by chute
- extracted_frames/by_camera/     # Frames organized by camera angle
- metadata/                       # Extraction metadata and logs
        


In [ ]:

# Multi-Camera Fall Dataset Preprocessing Pipeline
import cv2
import numpy as np
from pathlib import Path
import json
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

class MultiCameraPreprocessor:
    def __init__(self, frames_root, output_root, target_size=(224, 224)):
        """
        Initialize Multi-Camera Preprocessor

        Args:
            frames_root: Path to extracted frames directory
            output_root: Path to output directory for preprocessed data
            target_size: Target image size (height, width)
        """
        self.frames_root = Path(frames_root)
        self.output_root = Path(output_root)
        self.target_size = target_size

        # Create output directories
        self.create_output_structure()

        # Initialize augmentation transforms
        self.setup_transforms()

    def create_output_structure(self):
        """Create organized output directory structure"""
        directories = [
            'preprocessed_frames/train/fall',
            'preprocessed_frames/train/no_fall',
            'preprocessed_frames/val/fall', 
            'preprocessed_frames/val/no_fall',
            'preprocessed_frames/test/fall',
            'preprocessed_frames/test/no_fall',
            'augmented_frames/fall',
            'augmented_frames/no_fall',
            'normalized_frames',
            'metadata',
            'statistics'
        ]

        for dir_path in directories:
            (self.output_root / dir_path).mkdir(parents=True, exist_ok=True)

    def setup_transforms(self):
        """Setup image augmentation and preprocessing transforms"""
        # Basic preprocessing (resize, normalize)
        self.basic_transform = A.Compose([
            A.Resize(height=self.target_size[0], width=self.target_size[1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet stats
        ])

        # Training augmentation (includes data augmentation)
        self.train_transform = A.Compose([
            A.Resize(height=int(self.target_size[0] * 1.1), width=int(self.target_size[1] * 1.1)),
            A.RandomCrop(height=self.target_size[0], width=self.target_size[1]),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.GaussianBlur(blur_limit=(3, 7), p=0.3),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
            A.RandomGamma(gamma_limit=(80, 120), p=0.3),
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        # Validation/test transform (no augmentation)
        self.val_transform = A.Compose([
            A.Resize(height=self.target_size[0], width=self.target_size[1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def load_and_preprocess_image(self, image_path, transform=None):
        """Load and preprocess a single image"""
        try:
            # Load image
            image = cv2.imread(str(image_path))
            if image is None:
                print(f"Warning: Could not load image {image_path}")
                return None

            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Apply transforms
            if transform is not None:
                transformed = transform(image=image)
                image = transformed['image']
            else:
                # Default resize if no transform specified
                image = cv2.resize(image, (self.target_size[1], self.target_size[0]))
                image = image.astype(np.float32) / 255.0

            return image

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            return None

    def calculate_dataset_statistics(self):
        """Calculate mean and std of the dataset for normalization"""
        print("Calculating dataset statistics...")

        all_pixels = []

        # Process fall frames
        fall_dir = self.frames_root / 'extracted_frames/fall'
        if fall_dir.exists():
            for img_path in tqdm(list(fall_dir.rglob('*.jpg')), desc="Processing fall images"):
                img = cv2.imread(str(img_path))
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (self.target_size[1], self.target_size[0]))
                    all_pixels.append(img.reshape(-1, 3))

        # Process no-fall frames  
        no_fall_dir = self.frames_root / 'extracted_frames/no_fall'
        if no_fall_dir.exists():
            for img_path in tqdm(list(no_fall_dir.rglob('*.jpg')), desc="Processing no-fall images"):
                img = cv2.imread(str(img_path))
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (self.target_size[1], self.target_size[0]))
                    all_pixels.append(img.reshape(-1, 3))

        if all_pixels:
            all_pixels = np.vstack(all_pixels).astype(np.float32) / 255.0

            mean = np.mean(all_pixels, axis=0)
            std = np.std(all_pixels, axis=0)

            stats = {
                'mean': mean.tolist(),
                'std': std.tolist(),
                'total_pixels': len(all_pixels),
                'image_size': self.target_size
            }

            # Save statistics
            stats_file = self.output_root / 'statistics/dataset_stats.json'
            with open(stats_file, 'w') as f:
                json.dump(stats, f, indent=2)

            print(f"Dataset statistics saved to {stats_file}")
            print(f"Mean: {mean}")
            print(f"Std: {std}")

            return stats
        else:
            print("No images found for statistics calculation")
            return None

    def create_train_val_test_split(self, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
        """Create train/validation/test splits"""

        # Collect all scenarios
        fall_scenarios = list(range(1, 23))  # scenarios 01-22
        no_fall_scenarios = list(range(23, 25))  # scenarios 23-24

        # Split fall scenarios
        np.random.shuffle(fall_scenarios)
        n_fall = len(fall_scenarios)
        fall_train_end = int(n_fall * train_ratio)
        fall_val_end = fall_train_end + int(n_fall * val_ratio)

        fall_train = fall_scenarios[:fall_train_end]
        fall_val = fall_scenarios[fall_train_end:fall_val_end]
        fall_test = fall_scenarios[fall_val_end:]

        # For no-fall scenarios (only 2), use them for testing
        no_fall_train = []
        no_fall_val = []
        no_fall_test = no_fall_scenarios

        splits = {
            'train': {
                'fall': fall_train,
                'no_fall': no_fall_train
            },
            'val': {
                'fall': fall_val,
                'no_fall': no_fall_val
            },
            'test': {
                'fall': fall_test,
                'no_fall': no_fall_test
            }
        }

        # Save split information
        split_file = self.output_root / 'metadata/data_splits.json'
        with open(split_file, 'w') as f:
            json.dump(splits, f, indent=2)

        print(f"Data splits saved to {split_file}")
        return splits

    def process_scenario_images(self, scenario_num, is_fall, split_type, transform):
        """Process all images for a specific scenario"""
        scenario_dir = self.frames_root / f'extracted_frames/{"fall" if is_fall else "no_fall"}'

        processed_count = 0

        # Find all scenario directories (scenario_XX_camY)
        scenario_pattern = f"scenario_{scenario_num:02d}_cam*"
        for scenario_cam_dir in scenario_dir.glob(scenario_pattern):
            if not scenario_cam_dir.is_dir():
                continue

            # Process all images in this scenario+camera combination
            for img_path in scenario_cam_dir.glob('*.jpg'):
                # Load and preprocess image
                processed_img = self.load_and_preprocess_image(img_path, transform)

                if processed_img is not None:
                    # Generate output filename
                    output_filename = f"{scenario_cam_dir.name}_{img_path.stem}.npy"

                    # Save processed image
                    label = 'fall' if is_fall else 'no_fall'
                    output_path = self.output_root / f'preprocessed_frames/{split_type}/{label}/{output_filename}'

                    np.save(output_path, processed_img)
                    processed_count += 1

        return processed_count

    def preprocess_all_data(self):
        """Preprocess all extracted frames"""
        print("Starting comprehensive preprocessing...")

        # Calculate dataset statistics
        stats = self.calculate_dataset_statistics()

        # Create data splits
        splits = self.create_train_val_test_split()

        # Process each split
        total_processed = 0
        processing_log = {}

        for split_name, split_data in splits.items():
            print(f"\nProcessing {split_name} split...")

            # Select appropriate transform
            if split_name == 'train':
                transform = self.train_transform
            else:
                transform = self.val_transform

            split_processed = 0
            processing_log[split_name] = {'fall': 0, 'no_fall': 0}

            # Process fall scenarios
            for scenario_num in tqdm(split_data['fall'], desc=f"{split_name} fall scenarios"):
                count = self.process_scenario_images(scenario_num, True, split_name, transform)
                split_processed += count
                processing_log[split_name]['fall'] += count

            # Process no-fall scenarios
            for scenario_num in tqdm(split_data['no_fall'], desc=f"{split_name} no-fall scenarios"):
                count = self.process_scenario_images(scenario_num, False, split_name, transform)
                split_processed += count
                processing_log[split_name]['no_fall'] += count

            total_processed += split_processed
            print(f"{split_name} split: {split_processed} images processed")

        # Save processing log
        log_file = self.output_root / 'metadata/preprocessing_log.json'
        with open(log_file, 'w') as f:
            json.dump(processing_log, f, indent=2)

        print(f"\nTotal images preprocessed: {total_processed}")
        print(f"Processing log saved to {log_file}")

        return processing_log

    def generate_augmented_data(self, augmentation_factor=3):
        """Generate additional augmented samples for training"""
        print(f"Generating augmented data (factor: {augmentation_factor})...")

        train_fall_dir = self.output_root / 'preprocessed_frames/train/fall'
        augmented_fall_dir = self.output_root / 'augmented_frames/fall'

        if not train_fall_dir.exists():
            print("No training fall data found. Run preprocess_all_data() first.")
            return

        # Heavy augmentation transform
        heavy_aug = A.Compose([
            A.Resize(height=int(self.target_size[0] * 1.2), width=int(self.target_size[1] * 1.2)),
            A.RandomCrop(height=self.target_size[0], width=self.target_size[1]),
            A.HorizontalFlip(p=0.7),
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
            A.GaussianBlur(blur_limit=(3, 7), p=0.5),
            A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=15, p=0.5),
            A.RandomGamma(gamma_limit=(70, 130), p=0.5),
            A.CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, p=0.3),
            A.MotionBlur(blur_limit=7, p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        augmented_count = 0

        for img_path in tqdm(list(train_fall_dir.glob('*.npy')), desc="Creating augmented samples"):
            # Load preprocessed image (denormalize first)
            img = np.load(img_path)

            # Denormalize for augmentation
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img = img * std + mean
            img = (img * 255).astype(np.uint8)

            # Generate multiple augmented versions
            for aug_idx in range(augmentation_factor):
                try:
                    augmented = heavy_aug(image=img)['image']

                    # Save augmented image
                    output_filename = f"aug{aug_idx}_{img_path.stem}.npy"
                    output_path = augmented_fall_dir / output_filename

                    np.save(output_path, augmented)
                    augmented_count += 1

                except Exception as e:
                    print(f"Error augmenting {img_path}: {e}")
                    continue

        print(f"Generated {augmented_count} augmented samples")
        return augmented_count

    def create_data_loaders_metadata(self):
        """Create metadata for data loading"""
        metadata = {
            'dataset_info': {
                'name': 'Multiple Cameras Fall Dataset',
                'total_scenarios': 24,
                'cameras_per_scenario': 8,
                'target_size': self.target_size,
                'classes': ['fall', 'no_fall'],
                'num_classes': 2
            },
            'preprocessing': {
                'normalization_mean': [0.485, 0.456, 0.406],
                'normalization_std': [0.229, 0.224, 0.225],
                'augmentation_applied': True,
                'target_image_size': self.target_size
            },
            'file_structure': {
                'train': 'preprocessed_frames/train/',
                'val': 'preprocessed_frames/val/',
                'test': 'preprocessed_frames/test/',
                'augmented': 'augmented_frames/'
            }
        }

        # Count files in each split
        for split in ['train', 'val', 'test']:
            split_dir = self.output_root / f'preprocessed_frames/{split}'
            metadata[f'{split}_counts'] = {}

            for class_name in ['fall', 'no_fall']:
                class_dir = split_dir / class_name
                if class_dir.exists():
                    count = len(list(class_dir.glob('*.npy')))
                    metadata[f'{split}_counts'][class_name] = count

        # Save metadata
        metadata_file = self.output_root / 'metadata/data_loader_metadata.json'
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"Data loader metadata saved to {metadata_file}")
        return metadata

# Example usage
if __name__ == "__main__":
    # Initialize preprocessor
    preprocessor = MultiCameraPreprocessor(
        frames_root="processed_output/",  # From frame extraction step
        output_root="preprocessed_output/",
        target_size=(224, 224)  # Standard input size for most models
    )

    # Run complete preprocessing pipeline
    print("Starting Multi-Camera Fall Dataset Preprocessing Pipeline")
    print("="*60)

    # Step 1: Preprocess all data with train/val/test splits
    processing_log = preprocessor.preprocess_all_data()

    # Step 2: Generate augmented data for better training
    augmented_count = preprocessor.generate_augmented_data(augmentation_factor=3)

    # Step 3: Create metadata for data loaders
    loader_metadata = preprocessor.create_data_loaders_metadata()

    print("\nPreprocessing pipeline completed successfully!")
    print("Ready for skeletal structure detection and model training.")


In [1]:
import cv2
import os
from pathlib import Path

# Root folder
root_folder = '/kaggle/input/multiple-cameras-fall-dataset/dataset/dataset/chute01'

# Folder to save extracted and preprocessed frames
output_folder = '/kaggle/working/processed_output/extracted_frames/falls/chute_01_cam3'
os.makedirs(output_folder, exist_ok=True)

# Function to preprocess frames (example: convert to grayscale and resize)
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (224, 224))
    return resized

# Extract and preprocess frames from videos in root folder
for root, dirs, files in os.walk(root_folder):
    for file in files:
        if file.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(root, file)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                # Preprocess frame
                preprocessed_frame = preprocess_frame(frame)
                # Save frame as image
                frame_filename = f'{file.split(".")[0]}_{frame_count:06d}.jpg'
                cv2.imwrite(os.path.join(output_folder, frame_filename), preprocessed_frame)
                frame_count += 1
            cap.release()

[mpeg4 @ 0x41253540] mcbpc damaged at 37 18
[mpeg4 @ 0x41253540] Error at MB: 865
[mpeg4 @ 0x40f09380] ac-tex damaged at 24 17
[mpeg4 @ 0x40f09380] Error at MB: 806
[mpeg4 @ 0x3fe9be00] ac-tex damaged at 13 23
[mpeg4 @ 0x3fe9be00] Error at MB: 1071
[mpeg4 @ 0x4104a040] ac-tex damaged at 12 17
[mpeg4 @ 0x4104a040] Error at MB: 794
[mpeg4 @ 0x40f09180] Error at MB: 1285
[mpeg4 @ 0x40f54ac0] ac-tex damaged at 8 12
[mpeg4 @ 0x40f54ac0] Error at MB: 560
[mpeg4 @ 0x40e51c00] ac-tex damaged at 3 19
[mpeg4 @ 0x40e51c00] Error at MB: 877
[mpeg4 @ 0x40f8dc40] ac-tex damaged at 42 21
[mpeg4 @ 0x40f8dc40] Error at MB: 1008


In [4]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def process_video_with_skeleton(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        out.write(frame)

    cap.release()
    out.release()

video_path = '/kaggle/input/multiple-cameras-fall-dataset/dataset/dataset/chute01/cam1.avi'  # Replace with actual file
output_path = '/kaggle/working/skeleton_output.avi'
process_video_with_skeleton(video_path, output_path)


W0000 00:00:1756746565.620625     127 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756746565.675208     127 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756746565.783370     128 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
[mpeg4 @ 0x17b4da00] ac-tex damaged at 3 19
[mpeg4 @ 0x17b4da00] Error at MB: 877


In [5]:
import cv2
import mediapipe as mp
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    """Calculate angle between three points a, b, c (each a tuple of x,y)."""
    a = (a[0], a[1])
    b = (b[0], b[1])
    c = (c[0], c[1])

    ab = (a[0] - b[0], a[1] - b[1])
    cb = (c[0] - b[0], c[1] - b[1])

    dot = ab[0]*cb[0] + ab[1]*cb[1]
    mag_ab = math.sqrt(ab[0]**2 + ab[1]**2)
    mag_cb = math.sqrt(cb[0]**2 + cb[1]**2)

    if mag_ab * mag_cb == 0:
        return 0

    angle = math.acos(dot/(mag_ab*mag_cb))
    return math.degrees(angle)

def is_fall(pose_landmarks, image_height):
    # Get key points: left shoulder, right shoulder, left hip, right hip
    left_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_hip = pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

    # Midpoints of shoulders and hips
    shoulder_mid_y = (left_shoulder.y + right_shoulder.y) / 2 * image_height
    hip_mid_y = (left_hip.y + right_hip.y) / 2 * image_height

    # Vertical distance between shoulders and hips (in pixels)
    vertical_dist = hip_mid_y - shoulder_mid_y

    # Approximate body angle wrt vertical (simplified as line between mid-shoulder and mid-hip)
    # Use image coordinates (x,y), y increases downwards
    shoulder_mid = ((left_shoulder.x + right_shoulder.x)/2, (left_shoulder.y + right_shoulder.y)/2)
    hip_mid = ((left_hip.x + right_hip.x)/2, (left_hip.y + right_hip.y)/2)

    # Angle with respect to vertical axis
    dx = hip_mid[0] - shoulder_mid[0]
    dy = hip_mid[1] - shoulder_mid[1]
    angle = abs(math.degrees(math.atan2(dy, dx)) - 90)  # Angle difference from vertical line

    # Heuristic fall detection thresholds
    # For example, large horizontal body (angle > 45 deg) or very small vertical distance can indicate fall
    if angle > 45 or vertical_dist < image_height * 0.1:
        return True
    return False

def process_video_with_fall_detection(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        fall_detected = False
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            fall_detected = is_fall(results.pose_landmarks, height)

            if fall_detected:
                cv2.putText(frame, 'Fall Detected', (30, 60), cv2.FONT_HERSHEY_SIMPLEX,
                            1.5, (0, 0, 255), 3)
            else:
                cv2.putText(frame, 'No Fall', (30, 60), cv2.FONT_HERSHEY_SIMPLEX,
                            1.5, (0, 255, 0), 3)

        out.write(frame)

    cap.release()
    out.release()

# Usage
video_path = '/kaggle/input/multiple-cameras-fall-dataset/dataset/dataset/chute01/cam2.avi'
output_path = '/kaggle/working/fall_detection_output.avi'
process_video_with_fall_detection(video_path, output_path)


W0000 00:00:1756746816.465691     136 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756746816.531230     139 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
[mpeg4 @ 0x1f80f540] ac-tex damaged at 42 21
[mpeg4 @ 0x1f80f540] Error at MB: 1008
